In [64]:
import sys
import pprint
import spotipy
import spotipy.util as util
from urllib.request import urlopen
from spotipy.oauth2 import SpotifyClientCredentials
from PIL import Image
from IPython.core.display import display, HTML
from functools import reduce
from collections import Counter
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId

## SETUP
pp = pprint.PrettyPrinter()
client = MongoClient('localhost', 27017)
db = client.server

client_credentials_manager = SpotifyClientCredentials(
    client_id='9c93bd032a4340b086b31bd30ec8f24c',
    client_secret='b893fd0a51d34e2399effa91c1026de7')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## AUX FUNCTIONS
def printTrackSimple(track):
    print('(' + track['id'] + ') - ' + track['name'] + ' - ' + track['artists'][0]['name'])    
    return

def printTrackFull(track):
    feature_raw = [feature for feature in features_list if feature['id'] == track['id']][0]
    feature = dict((key, feature_raw[key]) for key in (
        'acousticness',
        'danceability',
        'energy',
        'instrumentalness',
        'liveness',
        'speechiness',
        'valence'))
    feature_meta = dict((key, feature_raw[key]) for key in (
        'duration_ms',
        'id',
        'key',
        'loudness',
        'mode',
        'tempo',
        'time_signature'))
    display(HTML("""
         <table style='width: 100%;padding: 0px; margin: 0px'>
            <tr>
                <td style='width: 64px;padding: 0px'>
                    <img src='"""+track['album']['images'][-1]['url']+"""'>
                </td>
                <td style='text-align: left;width: 300px'>
                    <b>"""+track['name']+"""</b>
                    <br>
                    <font color='grey'>"""+track['artists'][0]['name']+"""</font>
                </td>""" +
                 ''.join([str("""<td style='text-align: left;writing-mode: tb-rl;transform: rotate(-180deg)'>
                    <div style="height:64px; overflow:hidden">
                        """)+str(feature_name)+str("""
                        <br>
                        <font color='darkgrey'>""") + str(feature_meta[feature_name]) + str("""</font>                        
                    </div>
                </td>""") for feature_name in feature_meta]) + 
                 ''.join([str("""<td style='text-align: left;writing-mode: tb-rl;transform: rotate(-180deg)'>
                    <div style="height:64px; overflow:hidden">
                        """)+str(feature_name)+str("""
                        <br>
                        <progress value='""") + str(feature[feature_name]) + str("""' max='1' style='height: 64px;transform: rotate(180deg);'></progress>                        
                    </div>
                </td>""") for feature_name in feature]) +
"""            </tr>
        </table>"""))
    return

In [255]:
## CODE
username = 'Felipe Vasconcelos'
scope = 'user-top-read user-library-read'
max_songs = 5000 #00
token = util.prompt_for_user_token(
    username,
    scope,
    client_id='9c93bd032a4340b086b31bd30ec8f24c',
    client_secret='b893fd0a51d34e2399effa91c1026de7',
    redirect_uri='http://localhost:8888/tree')

if token:
    #Set token to spotify library
    sp = spotipy.Spotify(auth=token)
    print('Loading saved tracks...')
    user_tracks = []
    for offset in range(0, max_songs, 50):
        results = sp.current_user_saved_tracks(50,offset)
        if(offset > results['total']):
            break
        user_tracks.extend([item['track'] for item in results['items']])
        print(len(user_tracks), end=' \b')    
    print(len(user_tracks))
    
    #print('Loading tracks features...')
    features_list = []
    #for index in range(0, len(user_tracks), 100):
    #    features_list.extend(sp.audio_features(tracks=[track['id'] for track in user_tracks[index:index+100]]))
    #    print(len(features_list), end='\r')   
    #print(len(features_list))
    
else:
    print("Can't get token for", username)

Loading saved tracks...
50 100 150 200 250 267 267


In [11]:
## DISCOVER NEW TRACKS (based on user's profile)

## ARTISTS
# Get all user's artists
all_user_artists = []
[[all_user_artists.append(artist['id'])
    for artist in artist_list]
        for artist_list in [track['artists'] for track in user_tracks]]
all_user_artists = set(all_user_artists)
print("User's unique artists: ", len(all_user_artists))

# Get related artists
print('Loading related artists...')
all_related_artists = []
for artist_id in all_user_artists:
    related_artists = sp.artist_related_artists(artist_id)
    all_related_artists.extend([artist['id'] for artist in related_artists['artists']])
    print(len(all_related_artists), end=' \b')
print(len(all_related_artists))
all_related_artists = set(all_related_artists)
print("User's unique related artists: ", len(all_related_artists))

User's unique artists:  129
Loading related artists...
20 40 60 80 100 120 140 160 180 200 220 240 260 280 300 300 320 340 360 380 400 420 440 460 480 500 520 540 540 560 580 600 620 640 660 680 700 720 740 760 780 800 820 840 860 880 900 920 940 960 980 1000 1020 1040 1060 1080 1100 1120 1140 1160 1180 1200 1220 1240 1260 1280 1300 1320 1340 1360 1380 1400 1420 1440 1460 1480 1500 1520 1540 1560 1580 1600 1620 1640 1660 1680 1700 1720 1740 1760 1780 1800 1820 1840 1860 1860 1880 1900 1920 1940 1960 1980 2000 2020 2040 2060 2080 2099 2119 2139 2159 2179 2199 2219 2239 2259 2279 2299 2319 2339 2359 2379 2399 2419 2439 2459 2459 2479 2499 2499
User's unique related artists:  1328


In [21]:
# Get all artist's top tracks
print('Loading artists tracks...')
# all_tracks = []
# i = 0
for artist_id in list(all_related_artists):
    i += 1
    artist_top_tracks = sp.artist_top_tracks(artist_id)
    #all_tracks.extend([(track['artists'][0]['name'], track['name'], track['id']) for track in artist_top_tracks['tracks']])
    all_tracks.extend(artist_top_tracks['tracks'])
    print('('+str(i)+'/'+str(len(all_related_artists))+')', len(all_tracks), end=' \b')
print('('+str(i)+'/'+str(len(all_related_artists))+')', len(all_tracks))
#all_tracks = set(all_tracks)
print("Unique tracks: ", len(all_tracks))
## ALBUMS
# Get top albums

# Get top track's albums

# Get album's tracks

## FEATURES
# Get features


Loading artists tracks...
(1273/1328) 12474 (1274/1328) 12484 (1275/1328) 12494 (1276/1328) 12504 (1277/1328) 12514 (1278/1328) 12524 (1279/1328) 12534 (1280/1328) 12544 (1281/1328) 12554 (1282/1328) 12564 (1283/1328) 12574 (1284/1328) 12584 (1285/1328) 12594 (1286/1328) 12604 (1287/1328) 12614 (1288/1328) 12624 (1289/1328) 12634 (1290/1328) 12644 (1291/1328) 12654 (1292/1328) 12664 (1293/1328) 12674 (1294/1328) 12684 (1295/1328) 12694 (1296/1328) 12704 (1297/1328) 12714 (1298/1328) 12724 (1299/1328) 12734 (1300/1328) 12744 (1301/1328) 12744 (1302/1328) 12754 (1303/1328) 12757 (1304/1328) 12767 (1305/1328) 12777 (1306/1328) 12787 (1307/1328) 12797 (1308/1328) 12807 (1309/1328) 12817 (1310/1328) 12827 (1311/1328) 12837 (1312/1328) 12838 (1313/1328) 12848 (1314/1328) 12858 (1315/1328) 12868 (1316/1328) 12878 (1317/1328) 12888 (1318/1328) 12898 (1319/1328) 12908 (1320/1328) 12918 (1321/1328) 12928 (1322/1328) 12938 (1323/1328) 12948 (1324

In [27]:
## MONGODB
for track in all_tracks:
    track_item = {
        'id': track['id'],
        'album_id': track['album']['id'],
        'artists_ids': [artist['id'] for artist in track['artists']],
        'disc_number': track['disc_number'],
        'duration_ms': track['duration_ms'],
        'explicit': track['explicit'],
        'external_ids': track['external_ids'],
        'external_urls': track['external_urls'],
        'href': track['href'],
        'is_local': track['is_local'],
        'name': track['name'],
        'popularity': track['popularity'],
        'preview_url': track['preview_url'],
        'track_number': track['track_number'],
        'type': track['type'],
        'uri': track['uri'],
    }
    print('.- ', end=' \b')
    db.tracks.update_one({'id' : track_item['id']}, {"$set": track_item}, upsert=True)

.-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  .-  

In [33]:
print(sp.current_user())

{'country': 'BR', 'display_name': 'Felipe Vasconcelos', 'email': 'fevasc_707@hotmail.com', 'external_urls': {'spotify': 'https://open.spotify.com/user/12151527942'}, 'followers': {'href': None, 'total': 56}, 'href': 'https://api.spotify.com/v1/users/12151527942', 'id': '12151527942', 'images': [{'height': None, 'url': 'https://platform-lookaside.fbsbx.com/platform/profilepic/?asid=804160369615244&height=200&width=200&ext=1545175660&hash=AeTL2uwdtdbSwk-X', 'width': None}], 'product': 'premium', 'type': 'user', 'uri': 'spotify:user:12151527942'}


In [34]:
tracks_ids = [track['id'] for track in user_tracks]
document = {
    'id': sp.current_user()['id'],
    'tracks_ids': tracks_ids
}
db.users.insert_one(document)

In [191]:
user = db.users.find_one({'id' : '12151527942'})
pp.pprint(user)

{'_id': ObjectId('5bf1dee2c98d32526706ad29'),
 'id': '12151527942',
 'tracks_ids': ['2ViLeJgYQkn9BLbnxE1v1t',
                '6Hmj7SrLRbreLVfVS7mV1S',
                '2Y47OACzKVe2csji3iXCa3',
                '7BkEHDQO5rpMd2Ns0YVZBA',
                '5tIBZrJsVv4J15KXC3pqKc',
                '3UPnExZwftJ1KfxtnGstLg',
                '4JoYS9ly7OE8U5eFjGMZyI',
                '7MPT7tW5BOr2hxnzaZY7xp',
                '3H4THxnaoVjvHLN7fuiuZa',
                '6msm3dj6PqsZWNccgXZZVC',
                '0bvgTYp3MEsWmG8imCpJZZ',
                '49Yd1W7vmTyGXosX6vSVWM',
                '0a0xmwa0ik3iBKf6SsIe14',
                '1ddeP7G76wscmc0RH5sjVA',
                '1eJiwwK3UMhgrclHWLfMVd',
                '4QK5WwYwZz7t5hmUDyIoqv',
                '7fs6csqE3EltCrESv6cn0J',
                '6yLIqXX9edg1x0HZS7cZEv',
                '38LJ70drtNdHTXjsmF6Vs0',
                '5zK1xKbtwY2yupcaBXVGO6',
                '5icLsBPRCR8VkN1mMVzxDB',
                '6fsLHa9LMHrfPIIzZrm6J6',
        

In [44]:
i = 0
for item in db.tracks.find({'features' : None}):
    i = i + 1
    print(i, item['id'], item['name'])
# db.tracks.delete_many({'features' : None})

1 28uBKQl2ajCugiUSakr07B Árvores Alheias
2 57fDcLa203GnDF9OYJd6mV Shake up Christmas 2011 (Official Coca-Cola Christmas Song)
3 40NUZ2BbJZojWa8MlKG9U7 Xangô
4 2SSBx4eTGc85WP6Fbmj25c A Gente Se Fode Bem Pra Caramba
5 1tq7tn2VWpee2pd8hGLmqX Luz Vermelha - Kiko Dinucci Remix
6 1KjZTOXc1SKEoaaYwpdlgL O Inferno Tem Sede
7 2omGHzMb4ofOqfqElChyhS No Escuro
8 7kTcWNAP2pTp4zUMlyYEG5 Desmonto Sua Cabeça
9 1XdUWJE7SRgW8rjjqnM7Q3 Chorei
10 4YZv1s1WyDIAOdylymwGbz Inferno Particular
11 6fBbBl20ZDOYNtBcb4MYeY Fear of Pop
12 2L8TnFDvyBTPxAkedc0NN6 Engasga gato/Casa barata
13 7leEGF8s6zUSxb5gw10Wj3 I'm Sorry
14 4jm0g62RhER54Ol26EH6CQ Wasn't Yours to Take
15 10L0hN9k16IaRWnzyGFFCo Not What You Wanted
16 4alnTdZ8lqsIbxfj65U14s A Chance
17 78WnNXeZZMFH9lwMs0oBrj I Wonder
18 1nn14sFr4QHeVy2PIF0yQ5 Bitch Please
19 4wIkaOvkA3dWZln7GxH3mP Alone
20 12qWUT298OHRj6kiarUMpY Radio Ga Ga
21 55FQw6oxQGWsoPYGcmv1Ia Lady Jane
22 4P9gV5VrCSj83SeZ4FhzeS Sonhos
23 6CgNWKD0quMc4H7UbJrsvd Chama da paixão
24 6DfMFEmRMO0rJUN

In [312]:
# PRINT DE UMA MUSICA QUALQUER
pp.pprint(db.tracks.find_one({'id' : '3pSTUZbp7J5jg2JHChTf21'}))

{'_id': ObjectId('5bd67760edae5e837b647126'),
 'album_id': '5pIlMNPZh4D9iJSoCfMzGi',
 'artists_ids': ['2ohBjgFT9V0LRDWO2wF9DJ'],
 'disc_number': 1,
 'duration_ms': 232200,
 'explicit': False,
 'external_ids': {'isrc': 'BRSGL7200015'},
 'external_urls': {'spotify': 'https://open.spotify.com/track/3pSTUZbp7J5jg2JHChTf21'},
 'features': {'acousticness': 0.772,
              'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3pSTUZbp7J5jg2JHChTf21',
              'danceability': 0.581,
              'duration_ms': 232200,
              'energy': 0.677,
              'id': '3pSTUZbp7J5jg2JHChTf21',
              'instrumentalness': 0.00468,
              'key': 9,
              'liveness': 0.129,
              'loudness': -7.428,
              'mode': 0,
              'speechiness': 0.0422,
              'tempo': 105.468,
              'time_signature': 4,
              'track_href': 'https://api.spotify.com/v1/tracks/3pSTUZbp7J5jg2JHChTf21',
              'type': 'audio_features',


In [185]:
# VENDO QUANTIDADE DE TRACKS NO BANCO
db.tracks.count_documents({})

90706

In [50]:
# COLOCANDO FEATURES NA MUSICAS

print('Loading tracks features...')
for index in range(0, len(all_tracks), 100):
    features_list.extend(sp.audio_features(tracks=[track['id'] for track in all_tracks[index:index+100]]))
    print(len(features_list), end='\b ')   
print(len(features_list))

Loading tracks features...
13285


In [51]:
## INSERINDO FEATURES NAS MUSICAS DO BANCO
print('Inserting tracks features...')
i = 0
j = 0
for features in features_list:
    i += 1
    #pp.pprint(features)
    if(features is not None):
        bla = db.tracks.update_one({'id' : features['id']}, {"$set": {'features': features}})
    else:
        j += 1
    print(i, end='\b ')   
print(i)
print('Tracks without features: ', j)

Inserting tracks features...
13285
Tracks without features:  1


In [317]:
pp.pprint(features_list[1485:1500])

[{'acousticness': 0.0159,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4sBuUaRokHP5NZao9t559V',
  'danceability': 0.659,
  'duration_ms': 218638,
  'energy': 0.87,
  'id': '4sBuUaRokHP5NZao9t559V',
  'instrumentalness': 0.28,
  'key': 10,
  'liveness': 0.123,
  'loudness': -4.904,
  'mode': 1,
  'speechiness': 0.0331,
  'tempo': 124.003,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/4sBuUaRokHP5NZao9t559V',
  'type': 'audio_features',
  'uri': 'spotify:track:4sBuUaRokHP5NZao9t559V',
  'valence': 0.44},
 {'acousticness': 0.173,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5deRG3zaTg5qzFe5Jm30vb',
  'danceability': 0.757,
  'duration_ms': 192317,
  'energy': 0.599,
  'id': '5deRG3zaTg5qzFe5Jm30vb',
  'instrumentalness': 0.00244,
  'key': 6,
  'liveness': 0.0847,
  'loudness': -5.081,
  'mode': 1,
  'speechiness': 0.0293,
  'tempo': 140.053,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/5deRG3zaTg5qzFe

In [141]:
# Full print
for track in user_tracks[0:10]:
    printTrackFull(track)

,Coisas Que Eu Sei Danni Carlos,duration_ms 232093,id 6zQTiw3M1p58eklHhCBWyi,key 8,loudness -14.312,mode 0,tempo 119.966,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Serei Luz Natiruts,duration_ms 307061,id 0vWNGOcd15MecWZIw0jKcE,key 0,loudness -6.87,mode 0,tempo 169.982,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Butterflies and Hurricanes Muse,duration_ms 301733,id 0xJLcjd0gaZct43xG1UlXS,key 9,loudness -8.56,mode 1,tempo 115.326,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Hysteria Muse,duration_ms 227440,id 7xyYsOvq5Ec3P4fr6mM9fD,key 9,loudness -4.852,mode 0,tempo 93.162,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Knights Of Cydonia Muse,duration_ms 366213,id 7ouMYWpwJ422jRcDASZB7P,key 11,loudness -5.301,mode 0,tempo 137.114,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,City Of Delusion Muse,duration_ms 288266,id 3Sno9FE8r2uz8QP0MtnTPL,key 2,loudness -5.158,mode 0,tempo 119.817,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Map Of The Problematique Muse,duration_ms 258066,id 5YXr4AGfUQpLSxtFSsKUh6,key 8,loudness -3.613,mode 1,tempo 125.04,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Supermassive Black Hole Muse,duration_ms 212440,id 3lPr8ghNDBLc2uZovNyLs9,key 7,loudness -3.727,mode 1,tempo 120.0,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Starlight Muse,duration_ms 240213,id 3skn2lauGk7Dx6bVIt5DVj,key 4,loudness -4.046,mode 1,tempo 121.61,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


,Dentro da Música II - Ao Vivo Natiruts,duration_ms 310400,id 5D5HuZt3z1y7JSoWPCZfF7,key 7,loudness -8.203,mode 0,tempo 103.976,time_signature 4,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence


In [ ]:
# Simple print
for track in user_tracks[0:10]:
    printTrackSimple(track)

In [5]:
# Extract ids
print([track['id'] for track in user_tracks[0:10]])

['6zQTiw3M1p58eklHhCBWyi', '0vWNGOcd15MecWZIw0jKcE', '0xJLcjd0gaZct43xG1UlXS', '7xyYsOvq5Ec3P4fr6mM9fD', '7ouMYWpwJ422jRcDASZB7P', '3Sno9FE8r2uz8QP0MtnTPL', '5YXr4AGfUQpLSxtFSsKUh6', '3lPr8ghNDBLc2uZovNyLs9', '3skn2lauGk7Dx6bVIt5DVj', '5D5HuZt3z1y7JSoWPCZfF7']


In [52]:
# Get user's top artists
top_artists = sp.current_user_top_artists(50, 0, 'long_term')
pp.pprint([artist['name'] for artist in top_artists['items']])

['Jorge Ben Jor',
 'Los Hermanos',
 'Skank',
 'Criolo',
 'Charlie Brown Jr.',
 'Caetano Veloso',
 'The xx',
 'Beirut',
 'Radiohead',
 'Novos Baianos',
 'Coldplay',
 'Cartola',
 'Arctic Monkeys',
 'Chico Buarque',
 'LCD Soundsystem',
 'Cássia Eller',
 'Vinícius de Moraes',
 'Nação Zumbi',
 'Gilberto Gil',
 'Vanessa Da Mata',
 'Os Paralamas Do Sucesso',
 'Imagine Dragons',
 'Engenheiros Do Hawaii',
 'Marcelo D2',
 'Red Hot Chili Peppers',
 'Cage The Elephant',
 'Toquinho',
 'Beach House',
 'Foster The People',
 'Maroon 5',
 'The Killers',
 'Legião Urbana',
 'Elis Regina',
 'Alec Troniq',
 'U2',
 'Capital Inicial',
 'Lane 8',
 'Os Mutantes',
 'Marisa Monte',
 'Marbert Rocel',
 'Raul Seixas',
 'Mauricio Detoni',
 'Alabama Shakes',
 'Cazuza',
 'The Fratellis',
 'alt-J',
 'Ira!',
 'Natiruts',
 'Bezerra Da Silva',
 'Charles Trenet']


In [56]:
# Get user's top tracks
top_tracks = sp.current_user_top_tracks(50, 0, 'long_term')
pp.pprint([track['name'] for track in top_tracks['items']])

['We Are Young (feat. Janelle Monáe)',
 'Nantes',
 'Scar Tissue',
 'Filho Maravilha',
 'Sutilmente - Ao Vivo',
 'Seven Nation Army',
 'Chove Chuva',
 'Jackie Tequila',
 'Tente Outra Vez',
 'Take It Easy My Brother Charles',
 'Whistle For The Choir',
 'Song 2',
 'Elephant Gun',
 'Tiro Ao Álvaro',
 'Hard To Beat',
 'Até Quando?',
 'Chão de Giz - Ao Vivo',
 'Sutilmente',
 'Além do Que Se Vê - Ao Vivo',
 'País Tropical',
 'Cai a Noite',
 'Summertime Sadness',
 'Conselho',
 "You Don't Know Me",
 'Mountain Sound',
 'Mais Ninguém',
 'Counting Stars',
 'Quero Ser Feliz Também - Ao Vivo',
 'Oba, Lá Vem Ela',
 'Budapest',
 'Ela Vai Voltar (Todos Os Defeitos De Uma Mulher Perfeita)',
 'Preciso Me Encontrar',
 'Riptide',
 'Velha Infancia - 2004 Digital Remaster;',
 'Geração Coca-Cola',
 'Mistério do Planeta',
 'Do I Wanna Know?',
 'Isso',
 'Greenwashing',
 'Avôhai - Ao Vivo',
 'Flor de Lis',
 'Queremos Saber - Show Luz Do Solo',
 'Apesar De Você',
 'O Vento',
 'Feel Good Inc',
 'A Flor - Ao Vivo',

In [55]:
# Get spotify categories
categories = sp.categories(limit=50, offset=0)
pp.pprint([category['name'] for category in categories['categories']['items']])

['Top Lists',
 'Mexico',
 'Latin',
 'Pop',
 'Decades',
 'Mood',
 'Sleep',
 'Focus',
 'Romance',
 'Happy Holidays',
 'Comedy',
 'Dinner',
 'Rock',
 'Party',
 'Indie',
 'Chill',
 'Folk & Acoustic',
 'K-Pop',
 'J-Tracks',
 'Gaming',
 'Kids & Family',
 'Trending',
 'Hip-Hop',
 'Jazz',
 'Classical',
 'Christian',
 'Spotify Singles',
 'Punk',
 'Electronic/Dance',
 'Workout',
 'R&B',
 'Metal',
 'Reggae',
 'Afro',
 'Soul',
 'Blues',
 'Funk',
 'Travel',
 'Made in Colombia',
 'Brazilian music',
 'French Pop',
 'Desi',
 'Country',
 'WHM',
 'Black history is now',
 'Arab']


In [60]:
# Get related artist from artist id
related_artists = sp.artist_related_artists('06cd30Cv9US973Ika84gDw')
pp.pprint([artist['name'] for artist in related_artists['artists']])

['Israel Novaes',
 'Maiara & Maraisa',
 'Cleber & Cauan',
 'Pedro Paulo & Alex',
 'George Henrique & Rodrigo',
 'Cristiano Araújo',
 'Edu Chociay',
 'Naiara Azevedo',
 'Felipe Araújo',
 'Jefferson Moraes',
 'Diego & Victor Hugo',
 'Henrique & Diego',
 'Bruninho & Davi',
 'Thaeme & Thiago',
 'Zé Neto & Cristiano',
 'Israel & Rodolffo',
 'Fernando & Sorocaba',
 'Marcos & Belutti',
 'Paula Mattos',
 'Gabriel Gava']


In [298]:
# Get artist's top tracks from artist id
artist_top_tracks = sp.artist_top_tracks('1m00QT2CkW6rHi4dly8rVN')
pp.pprint([track for track in artist_top_tracks['tracks']])

[{'album': {'album_type': 'album',
            'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1m00QT2CkW6rHi4dly8rVN'},
                         'href': 'https://api.spotify.com/v1/artists/1m00QT2CkW6rHi4dly8rVN',
                         'id': '1m00QT2CkW6rHi4dly8rVN',
                         'name': 'Mallu Magalhães',
                         'type': 'artist',
                         'uri': 'spotify:artist:1m00QT2CkW6rHi4dly8rVN'}],
            'external_urls': {'spotify': 'https://open.spotify.com/album/5rCOOZsVbPWwuAVKrA5Xug'},
            'href': 'https://api.spotify.com/v1/albums/5rCOOZsVbPWwuAVKrA5Xug',
            'id': '5rCOOZsVbPWwuAVKrA5Xug',
            'images': [{'height': 640,
                        'url': 'https://i.scdn.co/image/6dca61a15906cd18aa44596808d847b35a2aaee2',
                        'width': 640},
                       {'height': 300,
                        'url': 'https://i.scdn.co/image/8db7392f5630a3597eb74f6141dc62031ee9

In [58]:
# Get spotify available genre seeds
genre_seeds = sp.recommendation_genre_seeds()['genres']
pp.pprint(genre_seeds)

['acoustic',
 'afrobeat',
 'alt-rock',
 'alternative',
 'ambient',
 'anime',
 'black-metal',
 'bluegrass',
 'blues',
 'bossanova',
 'brazil',
 'breakbeat',
 'british',
 'cantopop',
 'chicago-house',
 'children',
 'chill',
 'classical',
 'club',
 'comedy',
 'country',
 'dance',
 'dancehall',
 'death-metal',
 'deep-house',
 'detroit-techno',
 'disco',
 'disney',
 'drum-and-bass',
 'dub',
 'dubstep',
 'edm',
 'electro',
 'electronic',
 'emo',
 'folk',
 'forro',
 'french',
 'funk',
 'garage',
 'german',
 'gospel',
 'goth',
 'grindcore',
 'groove',
 'grunge',
 'guitar',
 'happy',
 'hard-rock',
 'hardcore',
 'hardstyle',
 'heavy-metal',
 'hip-hop',
 'holidays',
 'honky-tonk',
 'house',
 'idm',
 'indian',
 'indie',
 'indie-pop',
 'industrial',
 'iranian',
 'j-dance',
 'j-idol',
 'j-pop',
 'j-rock',
 'jazz',
 'k-pop',
 'kids',
 'latin',
 'latino',
 'malay',
 'mandopop',
 'metal',
 'metal-misc',
 'metalcore',
 'minimal-techno',
 'movies',
 'mpb',
 'new-age',
 'new-release',
 'opera',
 'pagode',

In [70]:
# Extract user's top genres from user's top artists
top_artists = sp.current_user_top_artists(50, 0, 'long_term')
top_genres = [artist['genres'] for artist in top_artists['items']]
top_genres_2d = reduce(lambda x, y: x+y, top_genres)
filtered_top_genres = [x for x in top_genres_2d if x in genre_seeds]
counter = Counter(filtered_top_genres)
common_genre_list = [genre[0] for genre in counter.most_common(5)]
pp.pprint(common_genre_list)

['mpb', 'samba', 'rock', 'forro', 'pagode']


In [75]:
# Get spotify recommendations from genre seeds
top_genre_count = 3
recomendation_list = sp.recommendations(seed_genres=common_genre_list[0:top_genre_count], limit=50)
pp.pprint([track['artists'][0]['name'] + ' - ' + track['name'] for track in recomendation_list['tracks']])

['Kasabian - Ill Ray (The King)',
 'Paulinho Da Viola - Rumo dos ventos',
 'Bezerra Da Silva - Mulher Sem Alma',
 'Twenty One Pilots - Chlorine',
 'Diogo Nogueira - Queria Tanto Lhe Ver',
 'Timbalada - Sambaê',
 'Vanessa Da Mata - Fotografia',
 'Wilson Simoninha - Mareio',
 'Sambô - Suddenly I See - Ao Vivo',
 'Dire Straits - Brothers In Arms',
 'Terra Samba - Simples (Easy)',
 'Ana Carolina - Resta',
 'Os Originais Do Samba - Olha O Padilha',
 'Antônio Carlos Jobim - Vive para Mañana',
 'Queens of the Stone Age - I Sat By The Ocean',
 'Dropkick Murphys - The Lonesome Boatman',
 "Queen - Don't Stop Me Now - Remastered",
 "Bad Company - Feel Like Makin' Love - 2015 Remastered Version",
 'Five Finger Death Punch - Trouble',
 'Emílio Santiago - Pot-Pourri: Na Cadência do Samba (Que Bonito É) / É Luxo '
 'Só',
 'Jimi Hendrix - The Wind Cries Mary',
 'Pop Evil - Footsteps',
 'Thirty Seconds To Mars - Walk On Water',
 'Moraes Moreira - Feito Jorge Ser Amado',
 'Zeca Baleiro - Nada Além',
 'I

In [37]:
for i in range(50):
    print(top_artists['items'][i]['name'], '-', top_genres[i])
#pp.pprint(top_genres)

Muse - ['modern rock', 'permanent wave', 'piano rock', 'post-grunge', 'rock']
Gorillaz - ['alternative hip hop', 'art pop']
Snarky Puppy - ['contemporary jazz', 'jazz funk', 'jazz fusion']
Greta Van Fleet - ['modern blues rock', 'modern hard rock', 'rock']
Natiruts - ['brazilian reggae']
Red Hot Chili Peppers - ['alternative rock', 'funk metal', 'permanent wave', 'post-grunge', 'rock']
The Weeknd - ['canadian pop', 'pop', 'rap']
Daft Punk - ['electro', 'electronic', 'filter house']
Drake - ['canadian hip hop', 'canadian pop', 'hip hop', 'pop rap', 'rap']
Darren Korb - ['chamber pop', 'deep chiptune', 'scorecore', 'video game music']
Cacife Clandestino - ['brazilian electronica', 'brazilian hip hop', 'deep funk carioca']
Eminem - ['detroit hip hop', 'g funk', 'hip hop', 'rap']
Led Zeppelin - ['album rock', 'classic rock', 'hard rock', 'rock']
Years & Years - ['dance pop', 'gauze pop', 'pop', 'tropical house']
Lulu Santos - ['brazilian rock', 'mpb', 'rock gaucho']
Saint Motel - ['indie p

In [38]:
pp.pprint(counter)

Counter({'rock': 11,
         'mpb': 10,
         'pop': 5,
         'samba': 4,
         'reggae': 3,
         'metal': 2,
         'pagode': 2,
         'edm': 2,
         'electro': 1,
         'electronic': 1,
         'dancehall': 1,
         'house': 1,
         'sertanejo': 1})


In [ ]:
#RECOMENDADOR 2 - BUSCA POR ARTISTAS

#ok - Buscar artistas mais escutados
#ok - Listar generos desses artistas
#ok - Quebrar em palavras
#ok - Filtrar por generos que podem ser seed
#ok - Contar generos que mais repetem
#ok - Pegar top 5 generos e buscar 50 ou 100 recomendacoes
#ok - Buscar artistas de cada musica

#Avaliar
#ok - Peso - Quais artistas sao meus?
#ok - Peso - Quais artistas sao relacionados?
#ok - Peso - Qual a posicao do genero

#ok - Imprimir ranking

In [210]:
#Buscar artistas mais escutados
f_user_top_artists = sp.current_user_top_artists(50, 0, 'long_term')
pp.pprint([artist['name'] for artist in f_user_top_artists['items']])

# print(len(f_user_top_artists['items']))
#primeiro artista
# pp.pprint(f_user_top_artists['items'][0])

['Jorge Ben Jor',
 'Los Hermanos',
 'Skank',
 'Criolo',
 'Charlie Brown Jr.',
 'Caetano Veloso',
 'The xx',
 'Beirut',
 'Radiohead',
 'Cartola',
 'Novos Baianos',
 'Coldplay',
 'Arctic Monkeys',
 'Chico Buarque',
 'LCD Soundsystem',
 'Cássia Eller',
 'Vinícius de Moraes',
 'Nação Zumbi',
 'Gilberto Gil',
 'Vanessa Da Mata',
 'Os Paralamas Do Sucesso',
 'Imagine Dragons',
 'Engenheiros Do Hawaii',
 'Marcelo D2',
 'Red Hot Chili Peppers',
 'Cage The Elephant',
 'Toquinho',
 'Beach House',
 'Foster The People',
 'Maroon 5',
 'The Killers',
 'Legião Urbana',
 'Elis Regina',
 'Alec Troniq',
 'U2',
 'Capital Inicial',
 'Lane 8',
 'Os Mutantes',
 'Marisa Monte',
 'Marbert Rocel',
 'Raul Seixas',
 'Mauricio Detoni',
 'Alabama Shakes',
 'Cazuza',
 'The Fratellis',
 'alt-J',
 'Ira!',
 'Natiruts',
 'Bezerra Da Silva',
 'Charles Trenet']


In [130]:
#buscar seeds de generos possiveis
genre_seeds = sp.recommendation_genre_seeds()['genres']
genre_seeds.append('bossa')
genre_seeds.append('brazilian')
pp.pprint(genre_seeds)


['acoustic',
 'afrobeat',
 'alt-rock',
 'alternative',
 'ambient',
 'anime',
 'black-metal',
 'bluegrass',
 'blues',
 'bossanova',
 'brazil',
 'breakbeat',
 'british',
 'cantopop',
 'chicago-house',
 'children',
 'chill',
 'classical',
 'club',
 'comedy',
 'country',
 'dance',
 'dancehall',
 'death-metal',
 'deep-house',
 'detroit-techno',
 'disco',
 'disney',
 'drum-and-bass',
 'dub',
 'dubstep',
 'edm',
 'electro',
 'electronic',
 'emo',
 'folk',
 'forro',
 'french',
 'funk',
 'garage',
 'german',
 'gospel',
 'goth',
 'grindcore',
 'groove',
 'grunge',
 'guitar',
 'happy',
 'hard-rock',
 'hardcore',
 'hardstyle',
 'heavy-metal',
 'hip-hop',
 'holidays',
 'honky-tonk',
 'house',
 'idm',
 'indian',
 'indie',
 'indie-pop',
 'industrial',
 'iranian',
 'j-dance',
 'j-idol',
 'j-pop',
 'j-rock',
 'jazz',
 'k-pop',
 'kids',
 'latin',
 'latino',
 'malay',
 'mandopop',
 'metal',
 'metal-misc',
 'metalcore',
 'minimal-techno',
 'movies',
 'mpb',
 'new-age',
 'new-release',
 'opera',
 'pagode',

In [153]:
#Listar generos desses artistas

#Quebrar em palavras
top_genres = [artist['genres'] for artist in f_user_top_artists['items']]
# pp.pprint(top_genres)
top_genres_2d = reduce(lambda x, y: x+y, top_genres)
# pp.pprint(top_genres_2d)
top_genres_2d_split = [genre.split() for genre in top_genres_2d]
# pp.pprint(top_genres_2d_split)
top_genres_2d_one_word = reduce(lambda x, y: x+y, top_genres_2d_split)
# pp.pprint(top_genres_2d_one_word)


# todos os generos (para avaliar perda de generos)
# counter = Counter(top_genres_2d_one_word)
# common_genre_list = [genre[0] for genre in counter.most_common(5)]
# pp.pprint(counter)

# corrigindo generos relacionados
top_genres_2d_one_word = ['bossanova' if genre == 'bossa' else genre for genre in top_genres_2d_one_word]
top_genres_2d_one_word = ['brazil' if genre == 'brazilian' else genre for genre in top_genres_2d_one_word]


#Filtrar por generos que podem ser seed
filtered_top_genres = [x for x in top_genres_2d_one_word if x in genre_seeds]

#Contar generos que mais repetem
counter = Counter(filtered_top_genres)
pp.pprint(counter)

Counter({'rock': 68,
         'brazil': 28,
         'mpb': 27,
         'indie': 23,
         'pop': 18,
         'samba': 15,
         'bossanova': 13,
         'alternative': 9,
         'forro': 8,
         'house': 6,
         'folk': 5,
         'garage': 5,
         'pagode': 4,
         'latin': 4,
         'punk': 3,
         'funk': 2,
         'afrobeat': 1,
         'dance': 1,
         'electronic': 1,
         'metal': 1,
         'blues': 1,
         'techno': 1,
         'edm': 1,
         'disco': 1,
         'reggae': 1})


In [213]:
#Pegar top 5 generos e buscar 50 ou 100 recomendacoes
common_genre_list = [genre[0] for genre in counter.most_common(5)]
pp.pprint(common_genre_list)

# Get spotify recommendations from genre seeds
recomendation_list = sp.recommendations(seed_genres=common_genre_list, limit=100)
pp.pprint([track['artists'][0]['name'] + ' - ' + track['name'] for track in recomendation_list['tracks']])
# print(recomendation_list['tracks'][0]['artists'][0])                                                            
    

['rock', 'brazil', 'mpb', 'indie', 'pop']
['Wiz Khalifa - Pull Up (feat. Lil Uzi Vert)',
 'Dr. Dog - The Breeze',
 'Gorillaz - On Melancholy Hill',
 'The xx - Say Something Loving',
 'Nickelback - Animals',
 'Leandro Léo - Felicidade',
 'Gabriel Moura - O Perfume Da Nêga',
 'Arca - Piel',
 'Highly Suspect - Wolf',
 'Maria Rita - Bolero De Satã - Live At Credicard Hall, São Paulo / 2012',
 'Disclosure - Help Me Lose My Mind',
 'Living Colour - Cult of Personality',
 'Adelitas Way - Notorious',
 'Moraes Moreira - Cuidado Moreira',
 'Seu Jorge - Errare Humanum Est',
 'Ana Carolina - Resta',
 'Mallu Magalhães - Velha e Louca',
 'Stanaj - Romantic - NOTD Remix',
 'Five Finger Death Punch - Sham Pain',
 'Childish Gambino - 3005',
 'MiC LOWRY - Oh Lord',
 'Queens of the Stone Age - I Sat By The Ocean',
 'Green Day - American Idiot',
 'My Bloody Valentine - Only Shallow',
 'New Politics - One of Us',
 'Little Joy - Keep Me In Mind',
 'Paramore - Grudges',
 'Astrud Gilberto - The Girl From Ipan

In [321]:
#Avaliacao

#avalia se as musicas recomendadas estao no bd (+1 no rating) e se o artista esta no bd (+1 rating)
dict_music_id_rating = {}
artist_list = []
rating = 0
for i in range(len(recomendation_list['tracks'])):
    rating = 0
    
    track_id = recomendation_list['tracks'][i]['id']
    artist_id = recomendation_list['tracks'][i]['artists'][0]['id']
    
    #musica na base
    track = db.tracks.find_one({'id' : track_id})
    if (track is not None):
        rating = rating + 1
#     print(track)
    
    #artista na base
    artist = db.tracks.find_one({'artists_ids' : artist_id})
    if (artist is not None):
        rating = rating + 1
#     print(artist)
    
    artist_list.append(artist_id)
    dict_music_id_rating[track_id] = rating

#dicionario 'musica id': 'rating'
print(dict_music_id_rating)


{'6LTsvaebP6V9tJFvZxqi5M': 1, '33VReBh5WbMIOiJI1Ml4o8': 1, '0q6LuUqGLUiCPP1cbdwFs3': 2, '728zbo72kOoYdGR3AhNsyF': 2, '3rjM7GhxdVq1YySsHBs21i': 1, '6SDVHcpzR0UMcHigMjQWgI': 2, '00Jq06w6G8zTkkOPt43KqL': 0, '3klBAH0wi8hR1kvRmiHRvw': 0, '2S7R4trL4HITPIfvKUxcA4': 2, '211LiGFSXcgXOwrb8sWOgN': 1, '175cWIYsv1G8WSHjSV0vue': 2, '5e3YOg6fIkP0wD5TyxcHOH': 0, '5PZo9NMyNLxOlpEFoYo611': 0, '25WATieNAKnnoKkzxHZeog': 2, '0wtLyvkwd4gjXfkMbNvCYZ': 1, '4UVhLfIzcgHsFdHfZVFPgC': 1, '0UtoASDXmEeZklHfKlqz8s': 2, '7e6FvCvngX5job1PUYIIIL': 2, '0nqIqABZcujg05mkixJV60': 1, '3Z2sglqDj1rDRMF5x0Sz2R': 2, '1XUiFMK48Yy4fgds8l9aLH': 0, '7oXRMDUzBPekkLRTJhSGvC': 2, '6nTiIhLmQ3FWhvrGafw2zj': 2, '52UcjsM15hjCQAUbTW2hy1': 2, '3zSaWBoojpQGDPO2oIL80H': 2, '5ruFO5Cbjvljsh3ItsDmp8': 2, '3WKz5JDH0St3Smips7NlOM': 1, '1Sq5WIPVCs0fMODkLpFxZL': 1, '1060gzllf4b0UETAXisR5l': 0, '6wNeKPXF0RDKyvfKfri5hf': 2, '2dpaYNEQHiRxtZbfNsse99': 2, '3wUX7HpXS382s1DHTRouVk': 2, '7oGZAicScQt96OAW4AruYy': 2, '4QeoDcR16IHpmmgFGQDrCp': 2, '1tg4ajDHN3Cr

In [263]:
#busca os dados dos artista no spotify 

#Para ser mais eficiente, poderia ser buscado no bd e se nao existisse buscado no spotify
#teria que criar um documento no bd do tipo artista e não apenas referencia-lo no documento track

artists_info_1 = sp.artists(artist_list[0:50])
artists_info_2 = sp.artists(artist_list[50:100])
artists_info = artists_info_1['artists'] + artists_info_2['artists']

In [327]:
#Finalizando a avaliacao

#Cada genero do artista no tops generos do usuario (+1 no rating)

dict_ratings = {}
artist_id_genre_count = {} # dict de id_artista e quantidade de top generos do usuario
for i in range(len(artists_info)):
    
    artist_id = artists_info[i]['id']
    artist_genres = artists_info[i]['genres']
#     print(artist_genres)
    
    if len(artist_genres) != 0:
        artist_genres_2d_split = [genre.split() for genre in artist_genres]
#         print(artist_genres_2d_split)
        artist_genres_2d_one_word = reduce(lambda x, y: x+y, artist_genres_2d_split)
#         print(artist_genres_2d_one_word)
        artist_genres_2d_one_word = ['bossanova' if genre == 'bossa' else genre for genre in artist_genres_2d_one_word]
        artist_genres_2d_one_word = ['brazil' if genre == 'brazilian' else genre for genre in artist_genres_2d_one_word]
#         print(artist_genres_2d_one_word)
    
    filtered_by_user_top_genres = [x for x in artist_genres_2d_one_word if x in common_genre_list]
    filtered_by_user_top_genres = list(set(filtered_by_user_top_genres))
#     print(filtered_by_user_top_genres)
#     print(len(filtered_by_user_top_genres))
    
    artist_id_genre_count[artist_id] = len(filtered_by_user_top_genres)

# verificar se o dict de idArtista + qtd Genero e dict idMusica + rating estao ok
# print(artist_id_genre_count)    
# print(dict_music_id_rating)

for i in range(len(recomendation_list['tracks'])):
    
    track_id = recomendation_list['tracks'][i]['id']
    artist_id = recomendation_list['tracks'][i]['artists'][0]['id']

    #somando ratings para ter o rating total
    dict_ratings[track_id] = dict_music_id_rating[track_id] + artist_id_genre_count[artist_id]
    
#ordernar por maior pontuacao
import operator
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
ordenado = sorted(dict_ratings.items(), key=operator.itemgetter(1), reverse=True)

#top 10 musicas recomendadas
print(ordenado[0:10])
    
    

[('0UtoASDXmEeZklHfKlqz8s', 6), ('5ZKCUMzsQavEeo36k1NXcY', 6), ('3pSTUZbp7J5jg2JHChTf21', 6), ('25WATieNAKnnoKkzxHZeog', 5), ('52UcjsM15hjCQAUbTW2hy1', 5), ('3zSaWBoojpQGDPO2oIL80H', 5), ('5ruFO5Cbjvljsh3ItsDmp8', 5), ('17o7MI50D0SjBvlYsaHDqi', 5), ('5WJFxVcC39g56Zf7Knh7xl', 5), ('2ma8V5vEOfCuyqZ0V8HAq3', 5)]


[1, 2, 3, 4, 5, 6]
